<a href="https://colab.research.google.com/github/wizkers007/fooddetection/blob/main/Food_volume_detection_with_reference_object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch torchvision torchaudio opencv-python matplotlib
!pip install mediapipe opencv-python #Mediapipe library for detecting finger
!pip install ultralytics

import torch
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import math

# --- Step 1: Initialize Mediapipe hands ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

user_input = input("Enter your finger length in cm: ")
finlength = float(user_input)

hands = mp_hands.Hands(
    static_image_mode=False,     # For live webcam, use False
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# --- Load image ---
from google.colab import files
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = cv2.imread(img_path)


# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold the image (adjust threshold as needed)
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# Find all contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if len(contours) < 2:
    print("Not enough contours found!")
else:
    # Sort contours by area (largest first)
    contours_sorted = sorted(contours, key=cv2.contourArea, reverse=True)

    # Take the second largest contour
    second_largest_contour = contours_sorted[1]

    # Draw the second largest contour for visualization
    cv2.drawContours(img, [second_largest_contour], -1, (0, 255, 0), 2)

    # Calculate pixel area
    pixel_area = cv2.contourArea(second_largest_contour)
    print(f"Second largest contour area in pixels²: {pixel_area}")





results = hands.process(img_rgb)

if results.multi_hand_landmarks:
  for hand_landmarks in results.multi_hand_landmarks:
    # Draw landmarks on the frame
    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Get specific landmarks (Index finger)
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_base = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]

    # Convert normalized coordinates to pixel values
    h, w, c = frame.shape
    tip_x, tip_y = int(index_tip.x * w), int(index_tip.y * h)
    base_x, base_y = int(index_base.x * w), int(index_base.y * h)

    # Draw points for visualization
    cv2.circle(frame, (tip_x, tip_y), 5, (0,255,0), -1)
    cv2.circle(frame, (base_x, base_y), 5, (0,0,255), -1)

  cv2.imshow("Hand Tracking", frame)

  if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
    break


pixel_distance = math.sqrt((tip_x - base_x)**2 + (tip_y - base_y)**2)

real_length_cm = finlength

# Suppose pixel_distance corresponds to the full finger in pixels
# Then 1 pixel = real_length_cm / pixel_distance
scale = real_length_cm / pixel_distance

#Multiply food pixel area by scale ot get real pizel area
real_area_cm2 = pixel_area * (scale ** 2)#(Since Area = (length*scale)*(height*scale))
print(f"Second largest contour area in cm²: {real_area_cm2:.2f}")

# Display the image with contour
cv2.imshow("Second Largest Contour", img)



print(f"Estimated food height: {food_height_cm:.2f} cm")

# --- Display image with annotations ---
plt.imshow(img_rgb)
plt.title(f"Estimated food height: {food_height_cm:.2f} cm")
plt.axis('off')
plt.show()




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing insta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject